## Análisis exploratorio de datos - Properati

In [114]:
# magic function para hacer que los graficos de matplotlib se renderizen en el notebook.
%matplotlib inline

import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('default') # haciendo los graficos un poco mas bonitos xD
plt.rcParams['figure.figsize'] = (15, 5)

In [115]:
# carga de un data frame
propiedades = pd.read_csv('../datasets/properati-AR-2017-08-01-properties-sell.csv', sep=',', error_bad_lines=False, low_memory=False)
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 27 columns):
id                            187482 non-null object
created_on                    187482 non-null object
operation                     187482 non-null object
property_type                 187482 non-null object
place_name                    187447 non-null object
place_with_parent_names       187482 non-null object
country_name                  187482 non-null object
state_name                    187482 non-null object
geonames_id                   158318 non-null float64
lat-lon                       118761 non-null object
lat                           118761 non-null float64
lon                           118761 non-null float64
price                         154443 non-null float64
currency                      154441 non-null object
price_aprox_local_currency    154443 non-null float64
price_aprox_usd               154443 non-null float64
surface_total_in_m2           12502

In [116]:
propiedades.head(5)

,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,2012-09-19,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,93.0,360.360360,1290.322581,NaN,NaN,NaN,http://www.properati.com.ar/udh_venta_casa_mer...,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,https://thumbs4.properati.com/1/ufoyqwscVk8n3e...
1,a205c53378e7879cf7519659317fb410c060d20e,2012-10-22,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,220.0,1000.000000,1000.000000,NaN,NaN,NaN,http://www.properati.com.ar/20hi_venta_casa_me...,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,https://thumbs4.properati.com/5/I5RS_jAIUseo9H...
2,e06d6f864a0b91346594da4d27ceddcef0947ea6,2012-11-01,sell,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-38.0199509,-57.5265733",...,42.0,NaN,NaN,NaN,2.0,NaN,http://www.properati.com.ar/336o_venta_departa...,DTO. 2 AMB. C/ VISTA PLENA AL 6 X 3DOMITORIO ...,DTO. 2 AMB. C/ VISTA PLENA AL MAR,https://thumbs4.properati.com/5/VjDOdzm9n8uqZu...
3,a07927207d2a03de5f4ba7fd76f670d6285a3315,2012-11-26,sell,house,Sierra de los Padres,|Argentina|Buenos Aires Interior|General Pueyr...,Argentina,Buenos Aires Interior,3427958.0,"-37.9511934894,-57.7752885836",...,265.0,NaN,NaN,NaN,6.0,NaN,http://www.properati.com.ar/47hc_venta_terreno...,PROPIEDAD DE CATEGORIA CUENTA CON: 4 DORMITOR...,CHALET EN VENTA,https://thumbs4.properati.com/9/DA4GMdxXrthh5z...
4,9446052c949050359fb08b9f1ba83ba2eaa20c99,2012-12-08,sell,house,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6544492346,-58.4895603251",...,103.0,616.666667,1796.116505,NaN,5.0,NaN,http://www.properati.com.ar/4m4m_venta_casa_ma...,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,https://thumbs4.properati.com/6/DsNJjCb5_zIPV0...


In [117]:
prop_caba_gba = propiedades[propiedades['state_name'] == 'Capital Federal']

In [118]:
prop_gba = propiedades[propiedades['state_name'].str.contains('G.B.A.')]
prop_gba.head(5)

,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,2012-09-19,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,93.0,360.36036,1290.322581,NaN,NaN,NaN,http://www.properati.com.ar/udh_venta_casa_mer...,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,https://thumbs4.properati.com/1/ufoyqwscVk8n3e...
1,a205c53378e7879cf7519659317fb410c060d20e,2012-10-22,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,220.0,1000.00000,1000.000000,NaN,NaN,NaN,http://www.properati.com.ar/20hi_venta_casa_me...,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,https://thumbs4.properati.com/5/I5RS_jAIUseo9H...
5,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,2013-01-04,sell,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,"-34.429071,-58.795418",...,54.0,NaN,1796.296296,NaN,2.0,NaN,http://www.properati.com.ar/518a_venta_departa...,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,https://thumbs4.properati.com/8/bJn9FqcBRGi_8H...
6,535441f569473f6e5d687c38b27d9b9c6e69e284,2013-01-09,sell,store,San Isidro,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|,Argentina,Bs.As. G.B.A. Zona Norte,3428983.0,"-34.4789383,-58.5214371",...,NaN,NaN,NaN,NaN,1.0,NaN,http://www.properati.com.ar/52p4_venta_local_s...,"Venta - Local - Argentina, Buenos Aires, Zona ...",Local en Venta ubicado en San Isidro,https://thumbs4.properati.com/9/ZCK2RkrsymyN5s...
10,61f3789618edabeaa7b5fc06ff36112f8a06557f,2013-01-26,sell,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,"-34.439241,-58.798589",...,45.0,NaN,2222.222222,NaN,2.0,NaN,http://www.properati.com.ar/59ec_venta_departa...,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Pilar,https://thumbs4.properati.com/5/hiXoFGFHBiIhV2...


In [119]:
prop_caba_gba = propiedades[propiedades['state_name'].str.contains('Capital Federal') | propiedades['state_name'].str.contains('G.B.A.')]
prop_caba_gba.head(5)

,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,2012-09-19,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,93.0,360.360360,1290.322581,NaN,NaN,NaN,http://www.properati.com.ar/udh_venta_casa_mer...,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,https://thumbs4.properati.com/1/ufoyqwscVk8n3e...
1,a205c53378e7879cf7519659317fb410c060d20e,2012-10-22,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,220.0,1000.000000,1000.000000,NaN,NaN,NaN,http://www.properati.com.ar/20hi_venta_casa_me...,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,https://thumbs4.properati.com/5/I5RS_jAIUseo9H...
4,9446052c949050359fb08b9f1ba83ba2eaa20c99,2012-12-08,sell,house,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6544492346,-58.4895603251",...,103.0,616.666667,1796.116505,NaN,5.0,NaN,http://www.properati.com.ar/4m4m_venta_casa_ma...,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,https://thumbs4.properati.com/6/DsNJjCb5_zIPV0...
5,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,2013-01-04,sell,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,"-34.429071,-58.795418",...,54.0,NaN,1796.296296,NaN,2.0,NaN,http://www.properati.com.ar/518a_venta_departa...,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,https://thumbs4.properati.com/8/bJn9FqcBRGi_8H...
6,535441f569473f6e5d687c38b27d9b9c6e69e284,2013-01-09,sell,store,San Isidro,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|,Argentina,Bs.As. G.B.A. Zona Norte,3428983.0,"-34.4789383,-58.5214371",...,NaN,NaN,NaN,NaN,1.0,NaN,http://www.properati.com.ar/52p4_venta_local_s...,"Venta - Local - Argentina, Buenos Aires, Zona ...",Local en Venta ubicado en San Isidro,https://thumbs4.properati.com/9/ZCK2RkrsymyN5s...


In [120]:
prop_caba_gba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124746 entries, 0 to 187480
Data columns (total 27 columns):
id                            124746 non-null object
created_on                    124746 non-null object
operation                     124746 non-null object
property_type                 124746 non-null object
place_name                    124711 non-null object
place_with_parent_names       124746 non-null object
country_name                  124746 non-null object
state_name                    124746 non-null object
geonames_id                   97866 non-null float64
lat-lon                       75504 non-null object
lat                           75504 non-null float64
lon                           75504 non-null float64
price                         110203 non-null float64
currency                      110202 non-null object
price_aprox_local_currency    110203 non-null float64
price_aprox_usd               110203 non-null float64
surface_total_in_m2           89385 non

In [121]:
prop_barrio = prop_caba_gba['state_name'].value_counts()
prop_barrio

Capital Federal             44152
Bs.As. G.B.A. Zona Norte    39888
Bs.As. G.B.A. Zona Oeste    22571
Bs.As. G.B.A. Zona Sur      18135
Name: state_name, dtype: int64

In [122]:
def parsear_zona(row):
    mylist = row['place_with_parent_names'].split("|")
    return mylist[3]

prop_caba_gba.loc[:,'BARRIO'] = prop_caba_gba.apply(parsear_zona, axis=1)   
prop_caba_gba.head(5)

/home/lcabrera/orga-datos/local/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,BARRIO
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,2012-09-19,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,360.360360,1290.322581,NaN,NaN,NaN,http://www.properati.com.ar/udh_venta_casa_mer...,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,https://thumbs4.properati.com/1/ufoyqwscVk8n3e...,Merlo
1,a205c53378e7879cf7519659317fb410c060d20e,2012-10-22,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,1000.000000,1000.000000,NaN,NaN,NaN,http://www.properati.com.ar/20hi_venta_casa_me...,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,https://thumbs4.properati.com/5/I5RS_jAIUseo9H...,Merlo
4,9446052c949050359fb08b9f1ba83ba2eaa20c99,2012-12-08,sell,house,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6544492346,-58.4895603251",...,616.666667,1796.116505,NaN,5.0,NaN,http://www.properati.com.ar/4m4m_venta_casa_ma...,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,https://thumbs4.properati.com/6/DsNJjCb5_zIPV0...,Mataderos
5,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,2013-01-04,sell,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,"-34.429071,-58.795418",...,NaN,1796.296296,NaN,2.0,NaN,http://www.properati.com.ar/518a_venta_departa...,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,https://thumbs4.properati.com/8/bJn9FqcBRGi_8H...,Pilar
6,535441f569473f6e5d687c38b27d9b9c6e69e284,2013-01-09,sell,store,San Isidro,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|,Argentina,Bs.As. G.B.A. Zona Norte,3428983.0,"-34.4789383,-58.5214371",...,NaN,NaN,NaN,1.0,NaN,http://www.properati.com.ar/52p4_venta_local_s...,"Venta - Local - Argentina, Buenos Aires, Zona ...",Local en Venta ubicado en San Isidro,https://thumbs4.properati.com/9/ZCK2RkrsymyN5s...,San Isidro


In [123]:
prop_caba_gba['BARRIO'].value_counts().head(10)

Tigre              13594
La Matanza          7947
Morón               5926
Palermo             5383
Vicente López       5218
San Isidro          5011
Pilar               4701
La Plata            4326
Lomas de Zamora     3828
Belgrano            3796
Name: BARRIO, dtype: int64

In [124]:
prop_caba_gba['property_type'].value_counts()

apartment    68826
house        44258
PH            7045
store         4617
Name: property_type, dtype: int64

In [125]:
moneda = prop_caba_gba['currency'].value_counts()
moneda

USD    105677
ARS      4523
PEN         2
Name: currency, dtype: int64

In [159]:
data = prop_caba_gba.loc[:,['BARRIO', 'price_aprox_usd']]
data.dropna(inplace=True)
group_by_barrio = data.groupby('BARRIO').agg([np.mean, np.size])
group_by_barrio = group_by_barrio.loc[group_by_barrio[('price_aprox_usd','size')] > 50].sort_values(('price_aprox_usd','mean'), ascending=False)
group_by_barrio.head(10)

price_aprox_usd         
                             mean     size
BARRIO                                    
Puerto Madero       933102.405806   1030.0
San Isidro          453277.572763   4900.0
Recoleta            451190.720953   1889.0
                    401593.354604   2515.0
Retiro              395079.866257    350.0
Tigre               381874.766485  12795.0
Belgrano            378185.525543   3276.0
Palermo             372118.074596   4687.0
Nuñez               370172.824995    933.0
General Rodríguez   364753.685656    221.0

In [160]:
data_barrio = prop_caba_gba.loc[:,['place_name', 'price_aprox_usd']]
data_barrio.dropna(inplace=True)
group_by_place = data_barrio.groupby('place_name').agg([np.mean, np.size])
group_by_place = group_by_place.loc[group_by_place[('price_aprox_usd','size')] > 50].sort_values(('price_aprox_usd','mean'), ascending=False)
group_by_place.head(10)

price_aprox_usd        
                                        mean    size
place_name                                          
Palermo Chico                   1.084513e+06   270.0
Puerto Madero                   9.331024e+05  1030.0
Santa Barbara Barrio Cerrado    7.815383e+05   115.0
Acassuso                        7.100821e+05   431.0
Barrio Los Lagos                6.151161e+05    62.0
Mayling Club de Campo           5.429888e+05    89.0
Martínez                        4.724978e+05  1689.0
Capital Federal                 4.648952e+05  1800.0
Recoleta                        4.511907e+05  1889.0
Vicente López                   4.494558e+05   929.0

## Comunas

In [ ]:
# carga de un data frame
comunas = pd.read_csv('../Data-Scients/datasets/comunas.csv', sep=',', error_bad_lines=False, low_memory=False)
comunas.info()

In [ ]:
comunas

## Subtes

In [ ]:
# carga de un data frame
subtes = pd.read_csv('../Data-Scients/datasets/estaciones-de-subte.csv', sep=',', error_bad_lines=False, low_memory=False)
subtes.info()

In [ ]:
subtes.head(10)

## Ferrocarriles

In [ ]:
# carga de un data frame
ferrocarriles_caba = pd.read_csv('../Data-Scients/datasets/estaciones-de-ferrocarril.csv', sep=';', error_bad_lines=False, low_memory=False)
ferrocarriles_caba.info()

In [ ]:
ferrocarriles_caba.head(10)

## Espacios verdes públicos

In [ ]:
# carga de un data frame
espacio_verde_caba = pd.read_csv('../Data-Scients/datasets/espacio-verde-publico.csv', sep=',', error_bad_lines=False, low_memory=False)
espacio_verde_caba.info()

In [ ]:
espacio_verde_caba.head(10)

## Establecimientos educativos

In [ ]:
# carga de un data frame
establecimientos_educativos = pd.read_csv('../Data-Scients/datasets/establecimientos-educativos.csv', sep=';', error_bad_lines=False, low_memory=False)
establecimientos_educativos.info()

In [ ]:
establecimientos_educativos.head(5)